In [1]:
#from ring_environment import Environment
import os
import gzip
import math
import copy
import pickle
import random
import numpy as np
import networkx as nx

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.multiprocessing as mp

from multiprocessing import Queue, Lock

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from agent import Agent
from mind import Mind
from ring_environment import Environment

In [3]:
class Environment:
    def __init__(self, size, num_actions = 2, name = None, 
            max_iteration = 5000, num_agents = 20):
       
        node_num = self.size = size
        self.graph = nx.cycle_graph(node_num)
        self.nodes = list(self.graph.nodes)
        self.num_agents = num_agents
        input_size = 1
        self.mind = Mind(input_size, num_actions)

        #to do: sort mind function:
        #- inputs, outputs, handling each agent's individual mind

        #self.A_mind = Mind(input_size, num_actions, self.lock, Queue())
        #self.B_mind = Mind(input_size, num_actions, self.lock, Queue())
        #for segregation the agent_range defines the window that each agent can observe which is then input to that nn
        self.max_iteration = max_iteration

        #self.vals = [-1, 0, 1, 2]  #possible values for grid squares in segregation case
        #self.names_to_vals = {"void": -2, "A": -1, "free": 0, "B": 1, "prey": 2}
        #self.vals_to_names = {v: k for k, v in self.names_to_vals.items()}
        #self.vals_to_index = {-1: 0, 0: 1, 1: 2, 2: 3}

        self.crystal = np.zeros((max_iteration, node_num, 1))  #id  -should there be other info in this?
        #what actually is this crystal object? Is this what's being saved?

        self.history = []        #think about what quantities I want to save
        self.id_track = []       #how to define, intitialise and store them 
        self.records = []

        if name:
            self.name = name

        if not os.path.isdir(str(self.name)):
            #os.mkdir(str(self.name))
            #os.mkdir(str(self.name)+'/episodes')
            self.map, self.agents, self.loc_to_agent, self.id_to_agent = self._generate_map()
            #self._set_initial_states()
            #self.mask = self._get_mask()
            #self.crystal = np.zeros((max_iteration, nodes, 2)) # tr,  id
            #self.iteration = 0
        else:
            assert False, "There exists an experiment with this name."



    def _generate_map(self):
        #generates representation of environment in a given state
        #rewrite this function to give graph with a given number of agents on each node
        #where:
        # map = graph with n agents on each node (state of environment)
        # agents = agents
        # loc_to_agent = location of each agent   what is the structure of these dicionaries? 
        # id_to_agent = id of each agent
        # work out how to include mind networks
        map = np.zeros(self.size)   #index corresponds to graph node i.e. map[0] = 1 means there is one agent on node zero
        loc_to_agent = {}
        id_to_agent = {}
        agents = []
        idx = 0
        init_nodes = np.zeros(self.num_agents) #initialise agent locations (gives initial node loaction for each agent i.e. init_node[0] = 10 means agent 0 begins on node 10) 
        for j in range(0, self.num_agents):   #needs to be outside loop over nodes so doesn't change each time
            init_nodes[j] = np.random.choice(self.nodes)
            mind = self.mind
            agent = Agent(idx, (init_nodes[j]),mind)
            loc_to_agent[(init_nodes[j])] = agent
            id_to_agent[idx] = agent
            agents.append(agent)
            idx += 1

        for i in enumerate(map):
            #print(i)
            for x in range(0, self.num_agents):
                #print(i[1], init_nodes[x])
                if i[0] == init_nodes[x]:
                    
                    #print(int(i[1]))
                    map[i[0]] += 1
            
            #val = np.random.choice(self.vals, p=self.probs)   #val = number of agents on node i
                                                                #not a random choice - should this come after or within agent loop/assignation
                                                                #similar structure if val not == 0?
                                                                #how to assign number of agents to graph originally?

            #map[i] = val     #this is the 'value' of the grid square in the segregation problem being added to the map
                                    #this is what I need to change to graph nodes with given numbers of agents present
        return map, agents, loc_to_agent, id_to_agent
    
    def move(self, agent):
        #to do: convert these location coordinates to graph nodes
        (i) = loc = agent.get_loc()     #only need single index to correspond to single node 
        (i_n) = to = agent.get_decision()  #decision returns 0 for move left and 1 for move right
                
        self.map[i] -= 1         #remove an agent from the previous node
        self.map[i_n] += 1       #add this agent to the new node 
        agent.set_loc(to)         #set this new node location to the agent
        self.loc_to_agent[to] = agent        
        del self.loc_to_agent[loc]   #delete the previous entry in the dictionary
        
    def step(self, agent, act):
         #function for calculating the local reward of each agent cumulative and global entropy reward should be calculated somewhere else - mind?
        (i) = agent.get_loc() # current location - change to graph
        assert self.loc_to_agent[(i)]
        (di) = act     #action 
        (i_n) = self._add((i), (di))
        agent.set_decision((i_n))
        self.move(agent)
        
        if self.map[i_n] == 1.0:   #if agent has moved to a node where it is the only one present it receives a positive reward
            rew = 1                     
            assert rew != None                             
        elif self.map[i_n] > 1.0:  #if agent moves to a node where there are also other agents then it recieves a negatibve reward
            rew = -1
            assert rew != None
        else:
            assert rew != None     #raise assertion error if node value at agent location is zero
        done = False
        self.update_agent(agent, rew, done)
        agent.clear_decision()
        return rew
    
    def _add(self, loc, act):
        i = loc
        di = act  #action (0=left, 1=right)
        if di == 0:
            #move left
             if loc == 0.0:
                (i_n) = to = 99.0
            else:
                (i_n) = to = loc - 1   
        if di == 1:
            #move right
            if loc < 99.0:
                (i_n) = to = loc + 1
            else:
                (i_n)= to = 0.0
        return (i_n)


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 130)

In [2]:
env = Environment(100, 2, 'Test1', 5000, 20)

6.0


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [23]:
agent_state = env.agents[0].get_state()
print(agent_state)
action = env.agents[0].decide(agent_state)
print(action)
#print(env.move(env.agents[0]))

None
0


In [15]:
#map, agents, loc_to_agent, id_to_agent = env._generate_map()

In [16]:

print('map', env.map)

map [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0.]


In [17]:
print('agents', env.agents[0].get_loc())   #env.agents[3].get_id() returns 3 which is correct

#need to update location so that it is inputting actual agent location not the same number

agents 19.0


In [18]:
print('loc', env.loc_to_agent)

loc {19.0: <agent.Agent object at 0x7fa870f09570>, 11.0: <agent.Agent object at 0x7fa870f0bd00>, 88.0: <agent.Agent object at 0x7fa870f08220>, 61.0: <agent.Agent object at 0x7fa870f09960>, 81.0: <agent.Agent object at 0x7fa870f0ba60>, 30.0: <agent.Agent object at 0x7fa870f0b640>, 77.0: <agent.Agent object at 0x7fa870f08280>, 80.0: <agent.Agent object at 0x7fa870f08a30>, 78.0: <agent.Agent object at 0x7fa870f09d20>, 28.0: <agent.Agent object at 0x7fa870f0a800>, 40.0: <agent.Agent object at 0x7fa870f0ac80>, 24.0: <agent.Agent object at 0x7fa87152ca60>, 76.0: <agent.Agent object at 0x7fa87152c5e0>, 20.0: <agent.Agent object at 0x7fa87152c580>, 95.0: <agent.Agent object at 0x7fa87152c550>, 36.0: <agent.Agent object at 0x7fa87152c5b0>, 6.0: <agent.Agent object at 0x7fa87152c610>, 45.0: <agent.Agent object at 0x7fa87152c520>, 74.0: <agent.Agent object at 0x7fa87152c490>, 86.0: <agent.Agent object at 0x7fa87152c4f0>}


In [19]:
print('id', env.id_to_agent)

id {0: <agent.Agent object at 0x7fa870f09570>, 1: <agent.Agent object at 0x7fa870f0bd00>, 2: <agent.Agent object at 0x7fa870f08220>, 3: <agent.Agent object at 0x7fa870f09960>, 4: <agent.Agent object at 0x7fa870f0ba60>, 5: <agent.Agent object at 0x7fa870f0b640>, 6: <agent.Agent object at 0x7fa870f08280>, 7: <agent.Agent object at 0x7fa870f08a30>, 8: <agent.Agent object at 0x7fa870f09d20>, 9: <agent.Agent object at 0x7fa870f0a800>, 10: <agent.Agent object at 0x7fa870f0ac80>, 11: <agent.Agent object at 0x7fa87152ca60>, 12: <agent.Agent object at 0x7fa87152c5e0>, 13: <agent.Agent object at 0x7fa87152c580>, 14: <agent.Agent object at 0x7fa87152c550>, 15: <agent.Agent object at 0x7fa87152c5b0>, 16: <agent.Agent object at 0x7fa87152c610>, 17: <agent.Agent object at 0x7fa87152c520>, 18: <agent.Agent object at 0x7fa87152c490>, 19: <agent.Agent object at 0x7fa87152c4f0>}


In [ ]:
#now know what all of these outputs are I can move onto other functions tomorrow 

In [8]:
test_array = np.zeros(10)

In [9]:
test_array[5] += 1

In [10]:
test_array

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])